**How many COVID cases will there be?**

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

In [104]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 4.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

MISSING_INFECTION_RATE_LOW = 5
MISSING_INFECTION_RATE_HIGH = 30

RANDOM_CASE_FLUCTUATION_LOW = -4000
RANDOM_CASE_FLUCTUATION_HIGH = 8000

DAILY_FOURTH_WAVE_CHANCE = 0.6 / 365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

VACCINE_DELAY_DAYS_LOW = 7
VACCINE_DELAY_DAYS_HIGH = 28

NEWS_ARTICLE_DELAY_DAYS_LOW = -5 # Earlier than it ought to be based on mistaken reporting, projections, estimations
NEWS_ARTICLE_DELAY_DAYS_HIGH = 15

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0),
                                     np.round(700 * million * (1/30) * (1/12), 0),
                                     np.round(800 * million * (1/30) * (1/12), 0),
                                     np.round(900 * million * (1/30) * (1/12), 0),
                                     np.round(1000 * million * (1/30) * (1/12), 0),
                                     np.round(1100 * million * (1/30) * (1/12), 0)]

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 4.0

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-01-01'
CURRENT_COVID_CASES = 61.8 * million
CURRENT_COVID_GROWTH = 571428
CURRENT_COVID_DERIVATIVE = 4000
Q4_2020_START_COVID = 26.02 * million

In [105]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [106]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
q4_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
jan_covid_collector = []
feb_covid_collector = []
mar_covid_collector = []
apr_covid_collector = []
may_covid_collector = []
jun_covid_collector = []
jul_covid_collector = []
aug_covid_collector = []
sep_covid_collector = []
oct_covid_collector = []
nov_covid_collector = []
dec_covid_collector = []
dec_2020_vaccinated_collector = []
jan_vaccinated_collector = []
feb_vaccinated_collector = []
mar_vaccinated_collector = []
apr_vaccinated_collector = []
may_vaccinated_collector = []
jun_vaccinated_collector = []
jul_vaccinated_collector = []
aug_vaccinated_collector = []
sep_vaccinated_collector = []
oct_vaccinated_collector = []
nov_vaccinated_collector = []
dec_vaccinated_collector = []


for n in range(N_SCENARIOS):
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
        
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    missing_infection_rate = np.round(lognormal_sample(MISSING_INFECTION_RATE_LOW, MISSING_INFECTION_RATE_HIGH, 0.8), 1)
    if missing_infection_rate < 1:
        missing_infection_rate = 1
        
    vaccine_modifier = np.round(lognormal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.2:
        vaccine_modifier = 0.2
    
    vaccine_delay_days = np.round(normal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
    
    news_article_delay_days = np.round(normal_sample(NEWS_ARTICLE_DELAY_DAYS_LOW, NEWS_ARTICLE_DELAY_DAYS_HIGH, 0.8), 1)
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Vaccine modifier: {}'.format(vaccine_modifier))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Missing infection rate: {}'.format(missing_infection_rate))
        print('Vaccine delay days: {}'.format(vaccine_delay_days))
        print('News article delay days: {}'.format(news_article_delay_days))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2020, 12, 15) - timedelta(days = vaccine_delay_days) - timedelta(days = news_article_delay_days)).days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines * vaccine_modifier,
                                                     daily_vaccines * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor * (1 / monthly_growth_modifier)
        else:
            covid_growth += (CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * monthly_growth_modifier
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid += covid_growth + random_case_fluctuation
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            dec_2020_vaccinated = total_vaccinated
            q4_2020_covid = covid - Q4_2020_START_COVID
            if verbose:
                print('...Q4 2020 COVID: {}'.format(q4_2020_covid))
            y2020_covid = np.round(covid * missing_infection_rate, 0)
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                print('...Total 2020 COVID after missing transform: {}'.format(y2020_covid))
                
        if date == '2021-01-30':
            jan_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            jan_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-02-28':
            feb_covid = covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            feb_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            mar_covid = covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            q1_2021_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            mar_vaccinated = total_vaccinated
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-04-30':
            apr_vaccinated = total_vaccinated
            apr_covid = covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-05-31':
            may_vaccinated = total_vaccinated
            may_covid = covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-06-30':
            jun_vaccinated = total_vaccinated
            jun_covid = covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-07-31':
            jul_vaccinated = total_vaccinated
            jul_covid = covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-08-31':
            aug_vaccinated = total_vaccinated
            aug_covid = covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-09-30':
            sep_vaccinated = total_vaccinated
            sep_covid = covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-10-31':
            oct_vaccinated = total_vaccinated
            oct_covid = covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-11-30':
            nov_vaccinated = total_vaccinated
            nov_covid = covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-12-31':
            dec_vaccinated = total_vaccinated
            dec_covid = covid - nov_covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
   
    dec_7_2020_covid_collector.append(dec7_covid)
    q4_2020_covid_collector.append(q4_2020_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    jan_covid_collector.append(jan_covid)
    feb_covid_collector.append(feb_covid)
    mar_covid_collector.append(mar_covid)
    apr_covid_collector.append(apr_covid)
    may_covid_collector.append(may_covid)
    jun_covid_collector.append(jun_covid)
    jul_covid_collector.append(jul_covid)
    aug_covid_collector.append(aug_covid)
    sep_covid_collector.append(sep_covid)
    oct_covid_collector.append(oct_covid)
    nov_covid_collector.append(nov_covid)
    dec_covid_collector.append(dec_covid)
    dec_2020_vaccinated_collector.append(dec_2020_vaccinated)
    jan_vaccinated_collector.append(jan_vaccinated)
    feb_vaccinated_collector.append(feb_vaccinated)
    mar_vaccinated_collector.append(mar_vaccinated)
    apr_vaccinated_collector.append(apr_vaccinated)
    may_vaccinated_collector.append(may_vaccinated)
    jun_vaccinated_collector.append(jun_vaccinated)
    jul_vaccinated_collector.append(jul_vaccinated)
    aug_vaccinated_collector.append(aug_vaccinated)
    sep_vaccinated_collector.append(sep_vaccinated)
    oct_vaccinated_collector.append(oct_vaccinated)
    nov_vaccinated_collector.append(nov_vaccinated)
    dec_vaccinated_collector.append(dec_vaccinated)


-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 7.5
Vaccine modifier: 1.3
Peak growth on: 2021-02-22
Missing infection rate: 18.7
Vaccine delay days: 18.5
News article delay days: 15.2
Day 0: 2020-12-03
...Total vaccinated now 0 (+0)
...Total COVID cases now 62375662.0 (+573081.0 +2581 -0.0 *3.1)
Day 1: 2020-12-04
...Total vaccinated now 0 (+0)
...Total COVID cases now 62948001.0 (+574734.0 -2395 -0.0 *3.1)
Day 2: 2020-12-05
...Total vaccinated now 0 (+0)
...Total COVID cases now 63524168.0 (+576387.0 -220 -0.0 *3.1)
Day 3: 2020-12-06
...Total vaccinated now 0 (+0)
...Total COVID cases now 64106151.0 (+578040.0 +3943 -0.0 *3.1)
Day 4: 2020-12-07
...Total vaccinated now 0 (+0)
...Total COVID cases now 64687097.0 (+579693.0 +1253 -0.0 *3.1)
...COVID by 7 Dec: 64687097.0
Day 5: 2020-12-08
...Total vaccinated now 0 (+0)
...Total COVID cases now 65275709.0 (+581346.0 +7266 -0.0 *3.1)
Day 6: 2020-12-09
...Total vaccinated now 0 (+0)
...Total COVID cases now 65862530.0 (+582999.0 +3822 -0

...Total vaccinated now 1239791579.0 (+10163118.0)
...Total COVID cases now 128841262.0 (+0 -1696 -203262.0 *4.4)
Day 386: 2021-12-24
...Total vaccinated now 1249954697.0 (+10163118.0)
...Total COVID cases now 128844526.0 (+0 +3264 -203262.0 *4.4)
Day 387: 2021-12-25
...Total vaccinated now 1260117815.0 (+10163118.0)
...Total COVID cases now 128844473.0 (+0 -53 -203262.0 *4.4)
Day 388: 2021-12-26
...Total vaccinated now 1270280933.0 (+10163118.0)
...Total COVID cases now 128841603.0 (+0 -2870 -203262.0 *4.4)
Day 389: 2021-12-27
...Total vaccinated now 1280444051.0 (+10163118.0)
...Total COVID cases now 128840890.0 (+0 -713 -203262.0 *4.4)
Day 390: 2021-12-28
...Total vaccinated now 1290607169.0 (+10163118.0)
...Total COVID cases now 128837877.0 (+0 -3013 -203262.0 *4.4)
Day 391: 2021-12-29
...Total vaccinated now 1300770287.0 (+10163118.0)
...Total COVID cases now 128835493.0 (+0 -2384 -203262.0 *4.4)
Day 392: 2021-12-30
...Total vaccinated now 1310933405.0 (+10163118.0)
...Total COVID

...Total vaccinated now 697083088.0 (+8430386.0)
...Total COVID cases now 116102856.0 (+0 +2543 -168608.0 *4.5)
Day 314: 2021-10-13
...Total vaccinated now 705513474.0 (+8430386.0)
...Total COVID cases now 116094109.0 (+0 -8747 -168608.0 *4.5)
Day 315: 2021-10-14
...Total vaccinated now 713943860.0 (+8430386.0)
...Total COVID cases now 116090216.0 (+0 -3893 -168608.0 *4.5)
Day 316: 2021-10-15
...Total vaccinated now 722374246.0 (+8430386.0)
...Total COVID cases now 116089577.0 (+0 -639 -168608.0 *4.5)
Day 317: 2021-10-16
...Total vaccinated now 730804632.0 (+8430386.0)
...Total COVID cases now 116101531.0 (+0 +11954 -168608.0 *4.5)
Day 318: 2021-10-17
...Total vaccinated now 739235018.0 (+8430386.0)
...Total COVID cases now 116106468.0 (+0 +4937 -168608.0 *4.5)
Day 319: 2021-10-18
...Total vaccinated now 747665404.0 (+8430386.0)
...Total COVID cases now 116107699.0 (+0 +1231 -168608.0 *4.5)
Day 320: 2021-10-19
...Total vaccinated now 756095790.0 (+8430386.0)
...Total COVID cases now 11

Day 242: 2021-08-02
...Total vaccinated now 800016053.0 (+8285132.0)
...Total COVID cases now 91512243.0 (+0 +6978 -165703.0 *2.4)
Day 243: 2021-08-03
...Total vaccinated now 808301185.0 (+8285132.0)
...Total COVID cases now 91516550.0 (+0 +4307 -165703.0 *2.4)
Day 244: 2021-08-04
...Total vaccinated now 816586317.0 (+8285132.0)
...Total COVID cases now 91514851.0 (+0 -1699 -165703.0 *2.4)
Day 245: 2021-08-05
...Total vaccinated now 824871449.0 (+8285132.0)
...Total COVID cases now 91518447.0 (+0 +3596 -165703.0 *2.4)
Day 246: 2021-08-06
...Total vaccinated now 833156581.0 (+8285132.0)
...Total COVID cases now 91519901.0 (+0 +1454 -165703.0 *2.4)
Day 247: 2021-08-07
...Total vaccinated now 841441713.0 (+8285132.0)
...Total COVID cases now 91530853.0 (+0 +10952 -165703.0 *2.4)
Day 248: 2021-08-08
...Total vaccinated now 849726845.0 (+8285132.0)
...Total COVID cases now 91536314.0 (+0 +5461 -165703.0 *2.4)
Day 249: 2021-08-09
...Total vaccinated now 858011977.0 (+8285132.0)
...Total COVI

Day 170: 2021-05-22
...Total vaccinated now 24671413.0 (+361119.0)
...Total COVID cases now 140318012.0 (+80316.0 -4785 -7222.0 *2.3)
Day 171: 2021-05-23
...Total vaccinated now 25032532.0 (+361119.0)
...Total COVID cases now 140388268.0 (+72936.0 +4542 -7222.0 *2.3)
Day 172: 2021-05-24
...Total vaccinated now 25393651.0 (+361119.0)
...Total COVID cases now 140449621.0 (+65556.0 +3019 -7222.0 *2.3)
Day 173: 2021-05-25
...Total vaccinated now 25754770.0 (+361119.0)
...Total COVID cases now 140495744.0 (+58176.0 -4831 -7222.0 *2.3)
Day 174: 2021-05-26
...Total vaccinated now 26115889.0 (+361119.0)
...Total COVID cases now 140536911.0 (+50796.0 -2407 -7222.0 *2.3)
Day 175: 2021-05-27
...Total vaccinated now 26477008.0 (+361119.0)
...Total COVID cases now 140579445.0 (+43416.0 +6340 -7222.0 *2.3)
Day 176: 2021-05-28
...Total vaccinated now 26838127.0 (+361119.0)
...Total COVID cases now 140616522.0 (+36036.0 +8263 -7222.0 *2.3)
Day 177: 2021-05-29
...Total vaccinated now 27199246.0 (+36111

...

-
-
## SCENARIO 11/1000 ##
Growth derivative factor: 8.7
Vaccine modifier: 1.4
Peak growth on: 2021-01-14
Missing infection rate: 9.7
Vaccine delay days: 12.2
News article delay days: 17.2
...COVID by 7 Dec: 64673937.0
...Q4 2020 COVID: 52874193.0
...Total 2020 COVID: 78894193.0
...Total 2020 COVID after missing transform: 765273672.0
...Q1 2021 COVID: 40075807.0
-
-
## SCENARIO 21/1000 ##
Growth derivative factor: 20.7
Vaccine modifier: 0.7
Peak growth on: 2020-12-03
Missing infection rate: 5.4
Vaccine delay days: 16.0
News article delay days: -2.3
...COVID by 7 Dec: 64674876.0
...Q4 2020 COVID: 52459047.0
...Total 2020 COVID: 78479047.0
...Total 2020 COVID after missing transform: 423786854.0
...Q1 2021 COVID: 36325909.0
-
-
## SCENARIO 31/1000 ##
Growth derivative factor: 6.4
Vaccine modifier: 1.3
Peak growth on: 2020-12-03
Missing infection rate: 21.3
Vaccine delay days: 15.1
News article delay days: 7.6
...COVID by 7 Dec: 64684595.0
...Q4 2020 COVID: 52434202.0
...Total 2020 COVID

-
-
## SCENARIO 241/1000 ##
Growth derivative factor: 8.8
Vaccine modifier: 1.1
Peak growth on: 2021-02-12
Missing infection rate: 17.0
Vaccine delay days: 25.9
News article delay days: -3.4
...COVID by 7 Dec: 64663285.0
...Q4 2020 COVID: 52807830.0
...Total 2020 COVID: 78827830.0
...Total 2020 COVID after missing transform: 1340073110.0
...Q1 2021 COVID: 43447527.0
-
-
## SCENARIO 251/1000 ##
Growth derivative factor: 20.6
Vaccine modifier: 5.1
Peak growth on: 2020-12-03
Missing infection rate: 31.2
Vaccine delay days: 17.8
News article delay days: 8.1
...COVID by 7 Dec: 64656822.0
...Q4 2020 COVID: 52413879.0
...Total 2020 COVID: 78433879.0
...Total 2020 COVID after missing transform: 2447137025.0
...Q1 2021 COVID: 17100736.0
-
-
## SCENARIO 261/1000 ##
Growth derivative factor: 0.3
Vaccine modifier: 0.9
Peak growth on: 2020-12-03
Missing infection rate: 14.2
Vaccine delay days: 30.4
News article delay days: -6.7
...COVID by 7 Dec: 64891774.0
...Q4 2020 COVID: 52502512.0
...Total 202

-
-
## SCENARIO 471/1000 ##
Growth derivative factor: 1.8
Vaccine modifier: 0.6
Peak growth on: 2020-12-24
Missing infection rate: 30.5
Vaccine delay days: 26.4
News article delay days: -1.6
...COVID by 7 Dec: 64816301.0
...Q4 2020 COVID: 56364641.0
...Total 2020 COVID: 82384641.0
...Total 2020 COVID after missing transform: 2512731550.0
...Q1 2021 COVID: 57411231.0
-
-
## SCENARIO 481/1000 ##
Growth derivative factor: 17.1
Vaccine modifier: 1.1
Peak growth on: 2020-12-16
Missing infection rate: 13.4
Vaccine delay days: 8.8
News article delay days: 7.8
...COVID by 7 Dec: 64678536.0
...Q4 2020 COVID: 52426015.0
...Total 2020 COVID: 78446015.0
...Total 2020 COVID after missing transform: 1051176601.0
...Q1 2021 COVID: 31769675.0
-
-
## SCENARIO 491/1000 ##
Growth derivative factor: 17.9
Vaccine modifier: 1.4
Peak growth on: 2020-12-28
Missing infection rate: 6.6
Vaccine delay days: 24.3
News article delay days: 7.9
...COVID by 7 Dec: 64676590.0
...Q4 2020 COVID: 52681187.0
...Total 2020 

-
-
## SCENARIO 701/1000 ##
Growth derivative factor: 26.3
Vaccine modifier: 0.8
Peak growth on: 2021-02-11
Missing infection rate: 3.1
Vaccine delay days: 28.2
News article delay days: 3.0
...COVID by 7 Dec: 64674915.0
...Q4 2020 COVID: 52661367.0
...Total 2020 COVID: 78681367.0
...Total 2020 COVID after missing transform: 243912238.0
...Q1 2021 COVID: 46978864.0
-
-
## SCENARIO 711/1000 ##
Growth derivative factor: 18.6
Vaccine modifier: 4.9
Peak growth on: 2020-12-03
Missing infection rate: 13.8
Vaccine delay days: 21.2
News article delay days: -1.0
...COVID by 7 Dec: 64665414.0
...Q4 2020 COVID: 52560511.0
...Total 2020 COVID: 78580511.0
...Total 2020 COVID after missing transform: 1084411052.0
...Q1 2021 COVID: 14524869.0
-
-
## SCENARIO 721/1000 ##
Growth derivative factor: 16.0
Vaccine modifier: 2.1
Peak growth on: 2020-12-09
Missing infection rate: 10.2
Vaccine delay days: 1.0
News article delay days: -0.0
...COVID by 7 Dec: 64677107.0
...Q4 2020 COVID: 52234057.0
...Total 2020

-
-
## SCENARIO 931/1000 ##
Growth derivative factor: 21.5
Vaccine modifier: 1.0
Peak growth on: 2020-12-03
Missing infection rate: 18.2
Vaccine delay days: 14.2
News article delay days: -9.8
...COVID by 7 Dec: 64695163.0
...Q4 2020 COVID: 52335150.0
...Total 2020 COVID: 78355150.0
...Total 2020 COVID after missing transform: 1426063730.0
...Q1 2021 COVID: 29835424.0
-
-
## SCENARIO 941/1000 ##
Growth derivative factor: 23.7
Vaccine modifier: 0.6
Peak growth on: 2020-12-03
Missing infection rate: 4.6
Vaccine delay days: 19.9
News article delay days: 9.8
...COVID by 7 Dec: 64681822.0
...Q4 2020 COVID: 52426304.0
...Total 2020 COVID: 78446304.0
...Total 2020 COVID after missing transform: 360852998.0
...Q1 2021 COVID: 45124706.0
-
-
## SCENARIO 951/1000 ##
Growth derivative factor: 9.3
Vaccine modifier: 0.2
Peak growth on: 2020-12-03
Missing infection rate: 14.8
Vaccine delay days: 19.3
News article delay days: 0.3
...COVID by 7 Dec: 64678591.0
...Q4 2020 COVID: 52345279.0
...Total 2020 

## Total Worldwide COVID by 7 Dec

For calibration

In [107]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 64669018.25, 'mean': 64676990.0, 'upper75': 64686467.5}

## 2020 Total COVID Cases with Missing Case Estimation

For https://www.metaculus.com/questions/3892/how-many-infections-of-sars-cov-2-novel-coronavirus-will-be-estimated-to-have-occurred-worldwide-before-2021/

In [108]:
get_metaculus_range(y2020_covid_collector, minx=1000000)

{'lower25': 598085209.25,
 'mean': 989709657.5,
 'upper75': 1618450295.0,
 'minx': '<1.0%'}

## 2020 Q4 Recorded COVID Cases

For https://www.metaculus.com/questions/3767/how-many-new-cases-of-covid-19-in-the-4th-quarter-of-2020/

In [109]:
get_metaculus_range(q4_2020_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 52399942.25,
 'mean': 52544633.0,
 'upper75': 52738704.0,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## 2021 Q1 Recorded COVID Cases

For https://www.metaculus.com/questions/3768/how-many-new-cases-of-covid-19-in-the-1st-quarter-of-2021/

In [110]:
get_metaculus_range(q1_2021_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 23514300.25,
 'mean': 35009135.5,
 'upper75': 43552887.25,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## Vaccinations

In [111]:
months = ['Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021', 'Jul 2021',
          'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021']

collectors = [dec_2020_vaccinated_collector, jan_vaccinated_collector, feb_vaccinated_collector,
              mar_vaccinated_collector, apr_vaccinated_collector, may_vaccinated_collector,
              jun_vaccinated_collector, jul_vaccinated_collector, aug_vaccinated_collector,
              sep_vaccinated_collector, oct_vaccinated_collector, nov_vaccinated_collector,
              dec_vaccinated_collector]

for i, month in enumerate(months):
    rangex = get_metaculus_range(collectors[i])
    print('total vaccinated by EOM {} -- 25% low {}M mean {}M 75% high {}M'.format(month,
                                                                                   np.round(rangex['lower25'] / million, 1),
                                                                                   np.round(rangex['mean'] / million, 1),
                                                                                   np.round(rangex['upper75'] / million, 1)))

total vaccinated by EOM Dec 2020 -- 25% low 0.0M mean 0.0M 75% high 0.3M
total vaccinated by EOM Jan 2021 -- 25% low 2.0M mean 4.1M 75% high 8.9M
total vaccinated by EOM Feb 2021 -- 25% low 6.2M mean 11.6M 75% high 23.0M
total vaccinated by EOM Mar 2021 -- 25% low 18.6M mean 35.2M 75% high 68.3M
total vaccinated by EOM Apr 2021 -- 25% low 41.6M mean 78.8M 75% high 149.1M
total vaccinated by EOM May 2021 -- 25% low 76.8M mean 139.9M 75% high 262.5M
total vaccinated by EOM Jun 2021 -- 25% low 120.7M mean 218.8M 75% high 406.6M
total vaccinated by EOM Jul 2021 -- 25% low 174.6M mean 315.6M 75% high 594.8M
total vaccinated by EOM Aug 2021 -- 25% low 243.8M mean 445.4M 75% high 836.3M
total vaccinated by EOM Sep 2021 -- 25% low 356.0M mean 638.3M 75% high 1203.0M
total vaccinated by EOM Oct 2021 -- 25% low 509.9M mean 910.3M 75% high 1692.7M
total vaccinated by EOM Nov 2021 -- 25% low 680.7M mean 1202.1M 75% high 2238.5M
total vaccinated by EOM Dec 2021 -- 25% low 874.6M mean 1546.4M 75% hi

In [112]:
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
for i, month in enumerate(months):
    print('10M vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 10 * million for x in collectors[i]]) / len(collectors[i]))))

# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
print('-')
for i, month in enumerate(months):
    print('100M vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 100 * million for x in collectors[i]]) / len(collectors[i]))))

# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
print('-')
for i, month in enumerate(months):
    print('1B vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 1000 * million for x in collectors[i]]) / len(collectors[i]))))

10M vaccinated by EOM Dec 2020: 1.0%
10M vaccinated by EOM Jan 2021: 21.0%
10M vaccinated by EOM Feb 2021: 57.0%
10M vaccinated by EOM Mar 2021: 91.0%
10M vaccinated by EOM Apr 2021: 99.0%
10M vaccinated by EOM May 2021: >99.0%
10M vaccinated by EOM Jun 2021: >99.0%
10M vaccinated by EOM Jul 2021: >99.0%
10M vaccinated by EOM Aug 2021: >99.0%
10M vaccinated by EOM Sep 2021: >99.0%
10M vaccinated by EOM Oct 2021: >99.0%
10M vaccinated by EOM Nov 2021: >99.0%
10M vaccinated by EOM Dec 2021: >99.0%
-
100M vaccinated by EOM Dec 2020: <1.0%
100M vaccinated by EOM Jan 2021: <1.0%
100M vaccinated by EOM Feb 2021: 1.0%
100M vaccinated by EOM Mar 2021: 16.0%
100M vaccinated by EOM Apr 2021: 39.0%
100M vaccinated by EOM May 2021: 65.0%
100M vaccinated by EOM Jun 2021: 81.0%
100M vaccinated by EOM Jul 2021: 92.0%
100M vaccinated by EOM Aug 2021: 96.0%
100M vaccinated by EOM Sep 2021: 99.0%
100M vaccinated by EOM Oct 2021: >99.0%
100M vaccinated by EOM Nov 2021: >99.0%
100M vaccinated by EOM Dec 2

## 2021 Peak Month

For https://www.metaculus.com/questions/5432/month-with-peak-of-covid-19-cases-in-2021/

In [113]:
max_month_collector = []

for i in range(N_SCENARIOS):
    covid_months = [jan_covid_collector[i], feb_covid_collector[i], mar_covid_collector[i], apr_covid_collector[i],
                    may_covid_collector[i], jun_covid_collector[i], jul_covid_collector[i], aug_covid_collector[i],
                    sep_covid_collector[i], oct_covid_collector[i], nov_covid_collector[i], dec_covid_collector[i]]
    max_month = months[covid_months.index(max(covid_months)) + 1]
    max_month_collector.append(max_month)

pd.Series(max_month_collector).value_counts(normalize=True) * 100

Jan 2021    98.7
Mar 2021     0.7
Feb 2021     0.5
Jun 2021     0.1
dtype: float64